In [ ]:
# Correr para reiniciar Kernel y la sesion.
!kill -9 -1

# Proyecto Final (Image + Text + Multi-Modal (Image + Text))

### Curso Deep Learning y Redes Neuronales - MIIA4406_01

#### Universidad de los Andes

Integrantes:

|Nombre|Codigo|
|---------------------|---------------------|
|Saby Espinel|*201215868*|
|Diego Salazar|*201628925*| <br>
<br>
______________________________________________________________________________________________________________________

## Verificacion de uso de 'GPU'

In [16]:
# Verificar que se esta usando GPU
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17386240137765768407, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11285974221
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8471661676241955885
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

# >>> Librerias y Funciones

In [17]:
##### Para cargar en Jupyter y CoLab #####
!pip install tqdm
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from skimage import io
from tqdm import tqdm

# Text processing
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split

# Text 
from keras.utils import np_utils
from keras.layers import Embedding, LSTM
from keras.optimizers import RMSprop
from keras.callbacks import History

## Image
import cv2
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dropout, Dense, Input, BatchNormalization, merge, Merge
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model, Sequential

%matplotlib inline

In [ ]:
##### Solo para Colab #####
!pip install -U -q PyDrive
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

In [ ]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [20]:
# Control de parametros
def controlParam(varTrain, path, max_review_length, top_words):
  varTrain = varTrain
  path = path
  max_review_length = max_review_length
  top_words = top_words
  return varTrain, path, max_review_length, top_words

# Para text detecta lemmas
import nltk
nltk.download('wordnet')

def split_into_lemmas(text):
    from nltk.stem import WordNetLemmatizer
    wordnet_lemmatizer = WordNetLemmatizer()
    
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word) for word in words]
  
# Para text detecta stemming
def split_into_stem(text):
    stemmer = SnowballStemmer('english')
    text = text.lower()
    words = text.split()
    return [stemmer.stem(word) for word in words]
  

[nltk_data] Downloading package wordnet to /content/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


******************************************************************************************************************************************

# >>> Read dataTraining and dataTesting

In [137]:
##### Para CoLab #####
# Verificacion de contenido de carpetas en el Drive con su respectivo ID
#file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
file_list = drive.ListFile({'q': "'1gX_xAhoYwIwmOVRd33lAuf4pCM3cChB6' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: ImageColab.ipynb, id: 1QI4l_c2wejdKDavNVdmMz6DSP0es2Jf2
title: Resultados.xlsx, id: 1Lykn5EEY4RTyTesTN6f5d0LDm1AqIuGv
title: pred_genres_img_RF.csv, id: 1l734WEkdZIeCUxGqJw_i0czo1bfm12IR
title: Combinado.ipynb, id: 1l_nE9YS7I3lFqzqTT62X46Cm9rkb2Vwt
title: ~$Resultados.xlsx, id: 1jvUOMuytd6qatDNqWg0NKSd_idSORy75
title: ProcText (1).ipynb, id: 1k6E6N6jG6PCCwXzgoEKCskST6_HwCYAO
title: ProcText.ipynb, id: 1d6n-hMm0W-Y9nXuUmGbfEACnmFmcUA2Z
title: 0-CombinacionesText, id: 1Lk9Ek6fOIAV6v9ysYe8U8beBpvFEtbPfqVb6tyd-_I4
title: features.pkl, id: 1gT_mDY7-1O9nwvWjFQj2qaVNhfl0TPFw
title: Final Project.pptx, id: 17-MkUf4ltoGnoNHS8XhuI2PzG0Igd9YF
title: Final Project, id: 1PIyDb9Q8kx5U-ShQRAAn1dXlVLRIPLs-VtXERlbpPW0
title: TextColab.ipynb, id: 1sEAr0qGqVwDkmZJjw4o1UC3owfqykJma
title: Text_Image_DeepLearning.ipynb, id: 1Cqw5bxyGIg6Pm-8hCAFuBh8-T-lwu_oi
title: ExampleImgRF.ipynb, id: 1Pg2-wYIUr866PzKvUy8iXsstZT87uE6x
title: Resultados_Jupyter, id: 1ANK5CbRmFQ6w-jbSzkkUO_bdjnUyXB6I
title: Resulta

In [ ]:
##### Para CoLab #####
trainDrive = drive.CreateFile({'id': '1SKNGe6FFghASG38LjtUaoTrmozUibRCZ'})
trainDrive.GetContentFile('dataTraining.csv')
dataTraining=pd.read_csv("dataTraining.csv", index_col=0)

testDrive = drive.CreateFile({'id': '1JWWmvheWnofZl2zqY-7Gu2rJprceYB80'})
testDrive.GetContentFile('dataTesting.csv')
dataTesting=pd.read_csv("dataTesting.csv", index_col=0)

In [185]:
# Fijar parametros de control
varTrain, path, max_review_length, top_words = controlParam(varTrain = 150, # Numero de muestrar para usar
             path = "images/", # Ruta donde estan almacenadas las imagenes
             max_review_length = 1000, # Numero maximo de largo permitido
             top_words = 100) # Maximo de palabras permitidas

# Funcion para seleccionar una muestra de las imagenes de dataTraining y dataTesting
def sampleData(dataTrain, dataTest, varTrain, varTest):
  ix_train=np.random.choice(dataTrain.shape[0], varTrain)
  ix_test=np.random.choice(dataTest.shape[0], varTest)
  return dataTrain.iloc[ix_train], dataTest.iloc[ix_test]

newDataTraining, newDataTesting  = sampleData(dataTraining, dataTesting, varTrain=varTrain, varTest=30)

100%|██████████| 150/150 [00:00<00:00, 1360.58it/s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


((150, 5), (30, 3), (150, 256, 160, 1), 7895, (150, 24))

# Text

In [11]:
# Create count vectorizer with ngrams
#vect = CountVectorizer(analyzer=split_into_lemmas, ngram_range=(1, 2), max_features=1000, stop_words='english') # LEMMAS
#vect = CountVectorizer(analyzer=split_into_stem, ngram_range=(1, 2), max_features=1000, stop_words='english') # STEM
vect = CountVectorizer(analyzer="word", ngram_range=(1, 2), max_features=max_review_length, stop_words='english')
X_dtm = vect.fit_transform(newDataTraining['plot'])
X_dtm.shape

(500, 1000)

# >>> Read Images CoLab

In [12]:
!ls

datalab  dataTesting.csv  dataTraining.csv  nltk_data


# Gray Images

In [ ]:
##### Para CoLab #####
trainDrive = drive.CreateFile({'id': '1SKNGe6FFghASG38LjtUaoTrmozUibRCZ'})
trainDrive.GetContentFile('dataTraining.csv')
dataTraining=pd.read_csv("dataTraining.csv", index_col=0)

testDrive = drive.CreateFile({'id': '1JWWmvheWnofZl2zqY-7Gu2rJprceYB80'})
testDrive.GetContentFile('dataTesting.csv')
dataTesting=pd.read_csv("dataTesting.csv", index_col=0)

# Download a file based on its file ID.
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
##### Para CoLab #####
imagesGray = drive.CreateFile({'id': '1mcEsaLxakpcQqYZl1pe_qtIEh4FXj0Wx'})
imagesGray.GetContentFile('images_resize_gray.zip')


In [22]:
!ls 

datalab		 dataTraining.csv    images_resize_gray.zip
dataTesting.csv  images_resize_gray  nltk_data


In [ ]:
# Tome las primeras 1025 + 1055 imagenes
!unzip -o images_resize_gray.zip

In [27]:
# Fijar parametros de control
varTrain, path, max_review_length, top_words = controlParam(varTrain = 1000, # Numero de muestrar para usar
             path = "images/", # Ruta donde estan almacenadas las imagenes
             max_review_length = 1000, # Numero maximo de largo permitido
             top_words = 100) # Maximo de palabras permitidas

# Funcion para seleccionar una muestra de las imagenes de dataTraining y dataTesting
def sampleData(dataTrain, dataTest, varTrain, varTest):
  ix_train=np.random.choice(dataTrain.shape[0], varTrain)
  ix_test=np.random.choice(dataTest.shape[0], varTest)
  return dataTrain.iloc[ix_train], dataTest.iloc[ix_test]

newDataTraining, newDataTesting  = sampleData(dataTraining, dataTesting, varTrain=varTrain, varTest=30)

path='images_resize_gray/'
# Training
images_training = []
newIndexTrain = []
for i in tqdm(newDataTraining.index):
    #path ="C:/Users/da.salazarb/Google Drive (dasalazarb@unal.edu.co)/AppliedDeepLearningClass-master/finalProject/"
    #images_training.append(io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())
    images_training.append(io.imread(os.path.join(path, str(i) + '_resize_gray.jpeg')))
    newIndexTrain.append(i)
    #images_training.append(io.imread(os.path.join(path, str(i) + ".jpeg")).flatten())
    
images_training = np.stack(images_training)
images_training = images_training.reshape(images_training.shape[0],images_training.shape[1], images_training.shape[2], 1)
#images_training = images_training.reshape(images_training.shape[0], 1, images_training.shape[1], images_training.shape[2])
images_training = images_training.astype('float32')
images_training /= 255
    

    
newDataTraining = dataTraining.loc[newIndexTrain]
newDataTraining['genres'] = newDataTraining['genres'].map(lambda x: eval(str(x)))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(newDataTraining['genres'])
newDataTraining.shape, newDataTesting.shape, images_training.shape, dataTraining.shape[0], y_genres.shape

100%|██████████| 1000/1000 [00:00<00:00, 1678.16it/s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


((1000, 5), (30, 3), (1000, 256, 160, 1), 7895, (1000, 24))

In [28]:
path='images_resize_gray/'
# Testing
images_testing  = []
newIndexTest = []
for i in tqdm(dataTesting.index):
    #images_testing.append(io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())
    #images_testing.append(io.imread(os.path.join(path, str(i) + '_resize_gray.jpeg')).flatten())
    images_testing.append(io.imread(os.path.join(path, str(i) + '_resize_gray.jpeg')))
    newIndexTest.append(i)

images_testing = np.stack(images_testing)
#images_testing = images_testing.reshape(images_testing.shape[0],images_testing.shape[1], images_testing.shape[2], 1)
images_testing = images_testing.reshape(images_testing.shape[0], images_testing.shape[1], images_testing.shape[2], 1)
images_testing = images_testing.astype('float32')
images_testing /= 255

images_testing.shape, dataTesting.shape[0]

100%|██████████| 3383/3383 [00:02<00:00, 1622.15it/s]


((3383, 256, 160, 1), 3383)

# Color Images

In [ ]:
# Download a file based on its file ID.
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
##### Para CoLab #####
imagesColor = drive.CreateFile({'id': '1_Grjc2zdV2gHA7unPg_I7qTEdM6zl1Ok'})
imagesColor.GetContentFile('images_original.zip')

In [14]:
!ls

datalab  dataTesting.csv  dataTraining.csv  images_original.zip  nltk_data


In [ ]:
# Tome las primeras 1025 + 1055 imagenes
!unzip -o images_original.zip 

In [16]:
path='images/'
# Training
images_training = []
newIndexTrain = []
for i in tqdm(newDataTraining.index):
    #path ="C:/Users/da.salazarb/Google Drive (dasalazarb@unal.edu.co)/AppliedDeepLearningClass-master/finalProject/"
    #images_training.append(io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())
    #images_training.append(io.imread(os.path.join(path, str(i) + '_resize_gray.jpeg')))
    archivoImg = cv2.resize(io.imread(os.path.join(path, str(i) + ".jpeg")), (224, 224))
    if len(archivoImg.shape) == 3:
      images_training.append(archivoImg)
      newIndexTrain.append(i)
images_training = np.stack(images_training)
images_training.shape, dataTraining.shape[0], newDataTraining.shape

 74%|███████▍  | 370/500 [00:14<00:05, 25.92it/s]/usr/local/lib/python3.6/dist-packages/PIL/Image.py:2274: DecompressionBombWarning: Image size (97200000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
100%|██████████| 500/500 [00:20<00:00, 23.82it/s]


((500, 224, 224, 3), 7895, (500, 5))

In [17]:
path='images/'
# Training
images_testing = []
newIndexTest = []
for i in tqdm(dataTesting.index):
    #path ="C:/Users/da.salazarb/Google Drive (dasalazarb@unal.edu.co)/AppliedDeepLearningClass-master/finalProject/"
    #images_training.append(io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())
    #images_training.append(io.imread(os.path.join(path, str(i) + '_resize_gray.jpeg')))
    archivoImg = cv2.resize(io.imread(os.path.join(path, str(i) + ".jpeg")), (224, 224))
    if len(archivoImg.shape) == 2:
      archivoImg = cv2.cvtColor(archivoImg,cv2.COLOR_GRAY2RGB)
    images_testing.append(archivoImg)
    newIndexTest.append(i)
    #print(i)
images_testing = np.stack(images_testing)
images_testing.shape, dataTesting.shape[0]

  3%|▎         | 102/3383 [00:02<01:22, 39.74it/s]/usr/local/lib/python3.6/dist-packages/PIL/Image.py:2274: DecompressionBombWarning: Image size (97143904 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
 61%|██████    | 2049/3383 [01:10<00:46, 28.90it/s]/usr/local/lib/python3.6/dist-packages/PIL/Image.py:2274: DecompressionBombWarning: Image size (97200000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
 61%|██████    | 2067/3383 [01:13<00:47, 27.99it/s]100%|██████████| 3383/3383 [01:56<00:00, 28.99it/s]


((3383, 224, 224, 3), 3383)

# Create *'y'* Colab (img + text)

In [ ]:
newDataTraining = dataTraining.loc[newIndexTrain]

In [99]:
newDataTraining['genres'] = newDataTraining['genres'].map(lambda x: eval(str(x)))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(newDataTraining['genres'])
y_genres.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(350, 23)

# Deep Neural Networks Images

In [20]:
from keras.applications.vgg19 import VGG19
base_modelVGG19 = VGG19(weights='imagenet')
base_modelVGG19.compile(optimizer="adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

574717952/574710816 [==============================] - 22s 0us/step


In [100]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
base_modelInceptionResNetV2 = InceptionResNetV2(weights='imagenet')
base_modelInceptionResNetV2.compile(optimizer="adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

225214464/225209952 [==============================] - 13s 0us/step


In [ ]:
def preTrainedModel(base_model):
  x = base_model.output
  x = Dense(256)(x)
  x = Activation("relu")(x)
  x = Dropout(0.5)(x)
  x = Dense(24)(x)
  x = Activation("sigmoid")(x)
  model = Model(inputs=base_model.input, outputs=x)
  model.compile(optimizer="rmsprop", loss = "categorical_crossentropy", metrics = ["accuracy"])
  return model

In [39]:
#model = preTrainedModel(base_modelInceptionResNetV2)
model = preTrainedModel(base_modelVGG19)

# fits the model on batches with real-time data augmentation:
#model.fit_generator(train_datagen.flow(images_training, y_genres, batch_size=32), steps_per_epoch=len(images_training) / 32, epochs=5)

#model.fit(X_train, y_train_genres, epochs=5, validation_data=[X_test, y_test_genres])
model.fit(x=images_training, y=y_genres, epochs=5, batch_size=50)

Epoch 1/5
500/500 [==============================] - 7s 14ms/step - loss: 8.7805 - acc: 0.0280
Epoch 2/5
500/500 [==============================] - 6s 12ms/step - loss: 8.6554 - acc: 0.0660
Epoch 3/5
500/500 [==============================] - 6s 12ms/step - loss: 8.5428 - acc: 0.1000
Epoch 4/5
500/500 [==============================] - 6s 12ms/step - loss: 8.4036 - acc: 0.1080
Epoch 5/5
500/500 [==============================] - 6s 12ms/step - loss: 8.2653 - acc: 0.1440


# Prediction on dataTesting dataset

In [ ]:
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = model.predict(images_testing)
#y_pred_test_genres = modeltune.predict_proba(images_testing)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_genres_img_RF.csv', index_label='ID')

In [ ]:
files.download('pred_genres_img_RF.csv')

# Fine Tunning

In [ ]:
base_modelVGG19.summary()

In [43]:
def fineTune(base_model):
  x = base_model.output
  #x = Flatten()(x)
  x = Dense(256)(x)
  x = Activation("relu")(x)
  x = Dropout(0.5)(x)
  x = Dense(256)(x)
  x = Activation("relu")(x)
  x = Dropout(0.5)(x)
  x = Dense(256)(x)
  x = Activation("relu")(x)
  x = Dropout(0.5)(x)
  x = Dense(256)(x)
  x = Activation("relu")(x)
  x = Dropout(0.5)(x)
  x = Dense(24)(x)
  x = Activation("sigmoid")(x)
  model = Model(inputs=base_model.input, outputs=x)
  #model.compile(optimizer="adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
  return model

modeltune = fineTune(base_modelVGG19)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_modelVGG19.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
modeltune.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics = ["accuracy"])

# train the model on the new data for a few epochs
modeltune.fit(x=images_training, y=y_genres, epochs=5)

# Vamos a escoger las layers de base_model para entrenarlas.
for layer in modeltune.layers[:31]:
   layer.trainable = False
for layer in modeltune.layers[31:]:
   layer.trainable = True

# Re-compilamos el modelos para que estas modificaciones tengan efecto
# Usamos SGD y un learning rate bajo
from keras.optimizers import SGD
modeltune.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics = ["accuracy"])

# Entrenamos el modelo nuevamente Esta vez con todo 
#(esta vez ajustando los 2 primeros bloques de inicio junto a las capas superiores de Dense)
modeltune.fit(x=images_training, y=y_genres, epochs=5)

Epoch 1/5
500/500 [==============================] - 7s 14ms/step - loss: 8.2620 - acc: 0.1660
Epoch 2/5
500/500 [==============================] - 5s 11ms/step - loss: 7.8114 - acc: 0.2000
Epoch 3/5
500/500 [==============================] - 5s 11ms/step - loss: 7.8108 - acc: 0.1980
Epoch 4/5
500/500 [==============================] - 5s 11ms/step - loss: 7.7641 - acc: 0.1740
Epoch 5/5
500/500 [==============================] - 5s 11ms/step - loss: 7.7588 - acc: 0.1660
Epoch 1/5
500/500 [==============================] - 6s 13ms/step - loss: 7.7262 - acc: 0.1700
Epoch 2/5
500/500 [==============================] - 5s 11ms/step - loss: 7.7313 - acc: 0.1680
Epoch 3/5
500/500 [==============================] - 5s 11ms/step - loss: 7.7324 - acc: 0.1700
Epoch 4/5
500/500 [==============================] - 5s 11ms/step - loss: 7.7256 - acc: 0.1720
Epoch 5/5
500/500 [==============================] - 5s 11ms/step - loss: 7.6905 - acc: 0.1700


# Prediction on dataTesting dataset

In [ ]:
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = modeltune.predict(images_testing)
#y_pred_test_genres = modeltune.predict_proba(images_testing)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_genres_img_RF.csv', index_label='ID')

In [ ]:
files.download('pred_genres_img_RF.csv')

# Convolutional Neural Network

In [ ]:
def convoNet(activation="tanh", optimizer="adam", drop=.4, neuron=132):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding="valid", data_format="channels_last", input_shape=(256, 160, 1)))
    model.add(Activation(activation))
    model.add(Dropout(drop))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(1, 1)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation(activation))
    model.add(Dropout(drop))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(1, 1)))

    #model.add(Conv2D(64, (3, 3)))
    #model.add(Activation(activation))
    #model.add(Dropout(drop))
    #model.add(BatchNormalization(axis=1))
    #model.add(MaxPooling2D(pool_size=(1, 1)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(neuron))
    model.add(Activation(activation))
    model.add(Dropout(0.5))
    model.add(Dense(24))
    model.add(Activation('sigmoid'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [19]:
images_training.shape, y_genres.shape

((200, 256, 160, 1), (200, 24))

In [32]:
model = convoNet()
model.fit(x=images_training, y=y_genres, epochs=5, batch_size=50)

Epoch 1/5


ResourceExhaustedError: ignored

# Other CNN + Merge

In [ ]:
rows, cols = 256, 160
def create_convnet():
    import keras
    input_shape = Input(shape=(rows, cols, 1))

    tower_1 = Conv2D(20, (100, 5), padding='same', activation='relu')(input_shape)
    tower_1 = MaxPooling2D((1, 11), strides=(1, 1), padding='same')(tower_1)

    tower_2 = Conv2D(20, (100, 7), padding='same', activation='relu')(input_shape)
    tower_2 = MaxPooling2D((1, 9), strides=(1, 1), padding='same')(tower_2)

    tower_3 = Conv2D(20, (100, 10), padding='same', activation='relu')(input_shape)
    tower_3 = MaxPooling2D((1, 6), strides=(1, 1), padding='same')(tower_3)

    merged = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1)
    merged = Flatten()(merged)

    out = Dense(200, activation='relu')(merged)
    out = Dense(24, activation='sigmoid')(out)

    model = Model(input_shape, out)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [37]:
model = create_convnet()

model.fit(images_training, y_genres, epochs=2, batch_size=50)

Epoch 1/2


ResourceExhaustedError: ignored

# Prediction on dataTesting dataset

In [ ]:
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = model.predict_proba(images_testing)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_genres_img_RF.csv', index_label='ID')

In [ ]:
files.download('pred_genres_img_RF.csv')

# Multi-modal Deep Learning Model

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=True,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(images_training)

model = convoNet()
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(images_training, y_genres,
                                 batch_size=100),
                    epochs=2,
                    workers=4)


# Prediction on dataTesting dataset

In [ ]:
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = model.predict_proba(images_testing)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_genres_img_RF.csv', index_label='ID')

In [ ]:
files.download('pred_genres_img_RF.csv')

# Prediction for multi-modal Neural Network

In [ ]:
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = model.predict_proba(X_test_dtm)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_genres_text_RF.csv', index_label='ID')